In [ ]:
%load_ext autoreload
%autoreload 2

: 

# Model Visualization

In [ ]:
import torch
import glob

import matplotlib.pyplot as plt
import viz

: 

### Load a pre-trained MAE model

In [ ]:
model = viz.prepare_model(chkpt_dir="out_mae_vit_base_xformers_scaled_dot_product_MLP_i128_p16_b256_e200", 
                          arch="mae_vit_base")

# model = viz.prepare_model(chkpt_dir="out_mae_vit_small_scaled_dot_product_i128_p16_b512_e200", 
#                           chkpt_name="40", # Optional
#                           arch="mae_vit_small")

: 

In [ ]:
# load an image
# img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRFQsDBSqgPKLHyunyQ0LJVDBXwzpnvWJqAOLPvVaUoA0xP-ZH4PPVCuyZtMllgEHSRTog&usqp=CAU'
# img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQSEHLflglbuqe5FfipsPHIQHOSmDX46YBMQIt2yiIZteSj6k_bnnVtAgNGrSWIYAovuzM&usqp=CAU'
# img_data = requests.get(img_url, stream=True).raw

img = viz.prepare_image('./images/0.jpg', model)
plt.rcParams["figure.figsize"] = [5, 5]
viz.show_image(torch.tensor(img))

: 

### Run MAE on the image

In [ ]:
viz.run_one_image(img, model, seed=None)
# run_one_image(img, model_mae, seed=2)

: 

In [ ]:

for img_path in glob.glob('./images/*.jpg'):
    img = viz.prepare_image(img_path, model)
    viz.run_one_image(img, model, seed=None, figsize=15)

: 